In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import datetime
print(f"last edit: {datetime.datetime.now()}")

last edit: 2025-07-28 07:38:55.529244


In [3]:
!nvidia-smi

Mon Jul 28 07:38:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
# Get data (10% of labels)
import zipfile

# Download data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

# Unzip the downloaded file
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip", "r")
zip_ref.extractall()
zip_ref.close()

--2025-07-28 07:38:55--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.10.207, 142.251.12.207, 64.233.170.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.10.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M  22.4MB/s    in 8.7s    

2025-07-28 07:39:04 (18.5 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [5]:
import os

for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f"there are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}")

there are 2 directories and 0 images in '10_food_classes_10_percent
there are 10 directories and 0 images in '10_food_classes_10_percent/test
there are 0 directories and 250 images in '10_food_classes_10_percent/test/ice_cream
there are 0 directories and 250 images in '10_food_classes_10_percent/test/sushi
there are 0 directories and 250 images in '10_food_classes_10_percent/test/steak
there are 0 directories and 250 images in '10_food_classes_10_percent/test/fried_rice
there are 0 directories and 250 images in '10_food_classes_10_percent/test/pizza
there are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_curry
there are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_wings
there are 0 directories and 250 images in '10_food_classes_10_percent/test/ramen
there are 0 directories and 250 images in '10_food_classes_10_percent/test/hamburger
there are 0 directories and 250 images in '10_food_classes_10_percent/test/grilled_salmon
there are 

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224,224)
BATCH_SIZE = 32

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Training images:")
train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                                          target_size = IMAGE_SHAPE,
                                                          batch_size = BATCH_SIZE,
                                                          class_mode ="categorical")

print("Testing images:")

test_data_10_percent = test_datagen.flow_from_directory(test_dir,
                                                          target_size = IMAGE_SHAPE,
                                                          batch_size = BATCH_SIZE,
                                                          class_mode ="categorical")

Training images:
Found 750 images belonging to 10 classes.
Testing images:
Found 2500 images belonging to 10 classes.


##**Setting up callback**

In [7]:
# Create tensorboard callback
def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name+ "/" + experiment_name + "/" +datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.tensorboard(
      lpg_dir=log_dir
  )
  print(f"Saving tensorboard log files to:{log_dir}")
  return tensorboard_callback

##**Creating models using tensorflow Hub**

In [8]:
## https://www.tensorflow.org/hub
## https://www.kaggle.com/models?tfhub-redirect=true&owner-type=organization&datatype=14102&task=16686
## https://github.com/paperswithcode

In [9]:
# Resnet 50 V2 feature vector

resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

# New: EfficientNetB0 feature vector (version 2)

efficientnet_url ="https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2"

These URLs link to a saved pretrained model on TensorFlow Hub.

When we use them in our model, the model will automatically be downloaded for us to use.

To do this, we can use the [`KerasLayer()`](https://www.tensorflow.org/hub/api_docs/python/hub/KerasLayer) model inside the TensorFlow hub library.

Since we're going to be comparing two models, to save ourselves code, we'll create a function `create_model()`. This function will take a model's TensorFlow Hub URL, instatiate a Keras Sequential model with the appropriate number of output layers and return the model.